# District Level Datasets

In [ ]:
district_emp = pd.read_csv('district_emp_data_full.csv')
district_emp = district_emp[['State/UT Name', 'District Name', 'Total Coal mining jobs (Direct + Indirect Jobs)', 
                             'Total Power plant  jobs (Direct + Indirect Jobs)', 'Total Coal Jobs (Direct + Indirect)', 
                             'Total Pensioners ', 'DMF USD (in Million $)', 'CSR  USD (in Million $)', 
                             'Mining CSR (in lakhs)', 'CSR Power Plant CSR (in lakhs)']]

district_emp.columns = ['State Name', 'District Name', 'Total Coal Mining Jobs', 
                             'Total Power Plant Jobs', 'Total Coal Jobs', 
                             'Total Pensioners', 'DMF USD', 'CSR USD', 
                             'Mining CSR INR', 'CSR Power Plant CSR INR'] # in million $, and lakh

district_emp = district_emp[district_emp['District Name'].isin([
    'Warangal Rural'
]) == False]

district_emp = district_emp.replace({
    'Aurangabad (Maha)': 'Aurangabad',
    'Bhadradri Kothagudem': 'Bhadradri',
    'Gondia': 'Gondiya',
    'Janjgir Champa': 'Janjgir-Champa',
    'Jayashankar Bhoopalpally': 'Jayashankar',
    'Korea': 'Koriya',
    'Kushi nagar': 'Kushinagar',
    'Lakhimpur Kheri': 'Lakhimpur',
    'Malda': 'Maldah',
    'Sahebganj': 'Sahibganj',
    'Sundergarh': 'Sundargarh',
    'Thoothukudi': 'Thoothukkudi',
    'Warangal Urban': 'Warangal',
    'Bangalore Urban': 'Bangalore',
    'Kutch': 'Kachchh',
    'Purulia': 'Puruliya',
    'Raigad': 'Raigarh',
    'Seraikela Kharsawan': 'Saraikela-Kharsawan',
    'South 24 Parganas': 'South Twenty Four Parganas',
    'North 24 Parganas': 'North Twenty Four Parganas',
    'Tiruvallur': 'Thiruvallur'
})

In [ ]:
district_dem = gpd.read_file('output.shp')[['statename', 'distname', 'st_areasha', 'distarea', 'totalpopul', 'geometry']]
district_dem.columns = ['State Name', 'District Name', 'State Area', 'District Area', 'Population', 'Geometry']

district_dem = district_dem[district_dem['District Name'].isin([
    'Imphal East', 'Warangal (R)'
]) == False]

district_dem = district_dem.replace({
    'NCT of Delhi': 'Delhi', 
    'Orissa': 'Odisha',
    'Ahmadabad': 'Ahmedabad',
    'Anugul': 'Angul',
    'Banas Kantha': 'Banaskantha',
    'Bara Banki': 'Barabanki',
    'Barddhaman': 'Bardhaman Purb',
    'Bid': 'Beed',
    'Buldana': 'Buldhana',
    'Chhatisgarh': 'Chhattisgarh',
    'Chittaurgarh': 'Chittorgarh',
    'Daman': 'Daman & Diu',
    'Diu': 'Daman & Diu',
    'Darjiling': 'Darjeeling',
    'Central': 'Central Delhi', 
    'East': 'East Delhi', 
    'West': 'West Delhi',
    'North': 'North Delhi', 
    'North East': 'North East Delhi',
    'North West': 'North West Delhi',
    'South': 'South Delhi',
    'South West': 'South West Delhi',
    'Gurgaon': 'Gurugram',
    'Hydrabad': 'Hyderabad',
    'Imphal West': 'Inner Manipur',
    'Jagatsinghapur':'Jagatsinghpur',
    'Jalor': 'Jalore',
    'Jhunjhunun': 'Jhunjhunu',
    'Kadapa(YSR)': 'Kadapa',
    'Kaimur (Bhabua)': 'Kaimur',
    'Kamrup Metropolitan': 'Kamrup',
    'Khandwa (East Nimar)': 'Khandwa',
    'Khargone (West Nimar)': 'Khargone',
    'Kodarma': 'Koderma',
    'Purulia': 'Puruliya',
    'Rae Bareli': 'Raebareli',
    'Sabar Kantha': 'Sabarkantha',
    'Warangal (U)': 'Warangal',
    'Hugli': 'Hooghly',
    'Mahrajganj': 'Maharajganj',
    'Bhiwani': 'Bhiwandi'
})

In [ ]:
district_votes = pd.read_csv('district_vote_data.csv')[['State', 'PC Name', 'Party', 'Margin %']]
district_votes.columns = ['State Name', 'District Name', 'Winning Party', 'Margin %']

district_votes = district_votes[district_votes['District Name'].isin([
    'Ahmedabad East', 'Arunachal East', 'Chennai South', 'Chennai North', 'Kolkata Dakshi', 'Maldaha Dakshi'
    'Mumbai North Central', 'Mumbai North East', 'Mumbai North West', 'Mumbai South Central'
]) == False]

district_votes = district_votes.replace({
    'Andhra Pradesh [2014 Onwards]': 'Andhra Pradesh', 
    'Bihar [2000 Onwards]': 'Bihar',
    'Delhi [1977 Onwards]': 'Delhi',
    'Madhya Pradesh [2000 Onwards]': 'Madhya Pradesh',
    'Uttar Pradesh [2000 Onwards]': 'Uttar Pradesh',
    'Orissa': 'Odisha',
    'Ahmedabad West': 'Ahmedabad',
    'Arunachal West': 'Arunachal',
    'Andaman And Nicobar Islands': 'Andaman & Nicobar Islands',
    'Bangalore Rura': 'Bangalore Rural',
    'Baramulla': 'Baramula',
    'Bhandara - Gon': 'Bhandara',
    'Bhiwani-Mahend': 'Bhiwani',
    'Chennai Centra': 'Chennai',
    'Chikkballapur': 'Chikkaballapura',
    'Dadra And Nagar Haveli': 'Dadra & Nagar Haveli',
    'Dakshina Kanna': 'Dakshina Kannada',
    'Daman And Diu': 'Daman & Diu',
    'Fatehgarh Sahi': 'Fatehgarh Sahib',
    'Gadchiroli-Chi': 'Gadchiroli',
    'Gautam Buddh N': 'Gautam Buddh Nagar',
    'Gurgaon': 'Gurugram',
    'Jaipur Rural': 'Jaipur',
    'Jajpur': 'Jajapur',
    'Jhalawar-Baran': 'Jhalawar',
    'Karauli-Dholpu': 'Karauli',
    'Kodarma': 'Koderma',
    'Kolkata Uttar': 'Kolkata',
    'Kushi Nagar': 'Kushinagar',
    'Maldaha Uttar': 'Maldaha',
    'Malkajgiri': 'Malkangiri',
    'Mandsour': 'Mandsaur',
    'Mumbai North': 'Mumbai Suburban',
    'Mumbai South': 'Mumbai',
    'Nabarangapur': 'Nabarangpur',
    'Nainital-Udham': 'Nainital',
    'Palamau': 'Palamu',
    'Patna Sahib': 'Patna',
    'Peddapalle': 'Peddapalli',
    'Ratnagiri - Si': 'Ratnagiri',
    'Sant Kabir Nag': 'Sant Kabir Nagar',
    'Thiruvananthap': 'Thiruvananthapuram',
    'Tiruchirappall': 'Tiruchirappalli',
    'Tonk-Sawai Mad': 'Tonk',
    'Udupi Chikmaga':'Udupi',
    'Yavatmal-Washi': 'Yavatmal'
})

district_votes['Margin %'] = district_votes['Margin %'].apply(lambda x: float(x.replace('%', '')))

In [ ]:
districts = district_dem.merge(district_emp, on=['State Name', 'District Name'], how='left')
districts = districts.merge(district_votes, on=['State Name', 'District Name'], how='left')
districts['Coal Job %'] = districts['Total Coal Jobs'] / districts['Population']
districts['Coal Pensioners %'] = districts['Total Pensioners'] / districts['Population']
districts.info()

In [ ]:
# export to pickle
filehandler = open("districts.pickle","wb")
pickle.dump(districts, filehandler)
filehandler.close()

### Explore Name Mismatches

In [ ]:
states_dem = set(np.unique(district_dem['State Name'].fillna('')))
states_emp = set(np.unique(district_emp['State Name'].fillna('')))
states_votes = set(np.unique(district_votes['State Name'].fillna('')))

district_dem_districts = set(np.unique(district_dem['District Name'].fillna('')))
district_emp_districts = set(np.unique(district_emp['District Name'].fillna('')))
district_votes_districts = set(np.unique(district_votes['District Name'].fillna('')))

In [ ]:
# States Missing Employment Data
states_dem - states_emp

In [ ]:
# Employment Data District Not Found
district_emp_districts - district_dem_districts

In [ ]:
# Districts Missing Employment Data
district_dem_districts - district_emp_districts

### Visualize District Data

In [ ]:
geoplot.choropleth(
    gpd.GeoSeries(districts['Geometry']), 
    hue=districts['Total Coal Jobs'],
    figsize=(12, 12), 
    legend=True,
    edgecolor='gray',
    linewidth=0.75
)

plt.title('Total Coal Jobs (Direct + Indirect)')
plt.show()

geoplot.choropleth(
    gpd.GeoSeries(districts['Geometry']), 
    hue=districts['Total Pensioners'], 
    figsize=(12, 12), 
    legend=True,
    edgecolor='gray',
    linewidth=0.75
)

plt.title('Total Pensioners')
plt.show()

geoplot.choropleth(
    gpd.GeoSeries(districts['Geometry']), 
    hue=districts['DMF USD'],
    figsize=(12, 12), 
    legend=True,
    edgecolor='gray',
    linewidth=0.75
)

plt.title('DMF USD (in Million $)')
plt.show()

geoplot.choropleth(
    gpd.GeoSeries(districts['Geometry']), 
    hue=districts['CSR USD'], 
    figsize=(12, 12), 
    legend=True,
    edgecolor='gray',
    linewidth=0.75
)

plt.title('CSR USD (in Million $)')
plt.show()

geoplot.choropleth(
    gpd.GeoSeries(districts['Geometry']), 
    hue= districts['Winning Party'].fillna(''),
#     scheme=districts['Margin %'], 
    figsize=(12, 12), 
    legend=True,
    edgecolor='gray',
    linewidth=0.75
)

plt.title('Winning Party')
plt.show()

# Targets

In [ ]:
wind_t_30 = {}
wind_t_30['Gujarat'] = 13.677
wind_t_30['Jammu and Kashmir'] = 0
wind_t_30['Arunachal Pradesh'] = 0
wind_t_30['West Bengal'] = 3.0
wind_t_30['Sikkim'] = 0
wind_t_30['Uttaranchal'] = 0
wind_t_30['Uttar Pradesh'] = 4.781
wind_t_30['Uttarakhand'] = 0.860
wind_t_30['Bihar'] = 3.0
wind_t_30['Jharkhand'] = 4.938
wind_t_30['Nagaland'] = 0
wind_t_30['Manipur'] = 0
wind_t_30['Mizoram'] = 0
wind_t_30['Tripura'] = 0
wind_t_30['Meghalaya'] = 0
wind_t_30['Punjab'] = 2.0
wind_t_30['Rajasthan'] = 13.130
wind_t_30['Himachal Pradesh'] = 0
wind_t_30['Andhra Pradesh'] = 8.000
wind_t_30['Daman and Diu'] = 0
wind_t_30['Maharashtra'] = 11.410
wind_t_30['Goa'] = 0.002
wind_t_30['Karnataka'] = 15.416
wind_t_30['Kerala'] = 0.788
wind_t_30['Puducherry'] = 0
wind_t_30['Tamil Nadu'] = 13.796
wind_t_30['Lakshadweep'] = 0
wind_t_30['Andaman and Nicobar Islands'] = 0
wind_t_30['Delhi'] = 0.096
wind_t_30['Dadra and Nagar Haveli'] = 0
wind_t_30['Chandigarh'] = 0
wind_t_30['Madhya Pradesh'] = 12.474
wind_t_30['Chhattisgarh'] = 5.171
wind_t_30['Haryana'] = 5.220
wind_t_30['Telangana'] = 8.482
wind_t_30['Odisha'] = 2.660
wind_t_30['Assam'] = 0

solar_t_30 = {}
solar_t_30['Jammu and Kashmir'] = 1.546
solar_t_30['Punjab'] = 6.0
solar_t_30['Haryana'] = 5.040
solar_t_30['Rajasthan'] = 18.042
solar_t_30['Delhi'] = 0.875
solar_t_30['Chandigarh'] = 0
solar_t_30['Himachal Pradesh'] = 0.500
solar_t_30['Uttar Pradesh'] = 11.500
solar_t_30['Uttarakhand'] = 4.0

solar_t_30['Goa'] = 1.025
solar_t_30['Gujarat'] = 14.26
solar_t_30['Madhya Pradesh'] = 12.923
solar_t_30['Chhattisgarh'] = 9.483
solar_t_30['Maharashtra'] = 12.506
solar_t_30['Daman and Diu'] = 0
solar_t_30['Dadra and Nagar Haveli'] = 0

solar_t_30['Andhra Pradesh'] = 8.1
solar_t_30['Telangana'] = 12.971
solar_t_30['Karnataka'] = 13.338
solar_t_30['Kerala'] = 2.625
solar_t_30['Tamil Nadu'] = 18.449
solar_t_30['Puducherry'] = 0

solar_t_30['Bihar'] = 10.431
solar_t_30['Jharkhand'] = 5.277
solar_t_30['Odisha'] = 10.971
solar_t_30['West Bengal'] = 8.077
solar_t_30['Sikkim'] = 0

solar_t_30['Assam'] = 0
solar_t_30['Manipur'] = 0
solar_t_30['Meghalaya'] = 0
solar_t_30['Nagaland'] = 0
solar_t_30['Mizoram'] = 0
solar_t_30['Tripura'] = 0
solar_t_30['Arunachal Pradesh'] = 6.870

solar_t_30['Lakshadweep'] = 0
solar_t_30['Andaman and Nicobar Islands'] = 0